<a href="https://colab.research.google.com/github/annikamarie/CalypsoAI/blob/main/CalypsoAI_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from __future__ import print_function, division
%tensorflow_version 1.15
import tensorflow as tf

from tensorflow.python.summary.writer.writer import FileWriter
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
from sklearn.metrics import precision_recall_fscore_support as score
from tensorflow.contrib.rnn import GRUCell, LSTMCell
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import sklearn as sk
import pandas as pd
import numpy as np
import pickle
import json
import time
import sys
import re
import os

tf.logging.set_verbosity(tf.logging.ERROR)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
sys.path.insert(0,'/content/gdrive/MyDrive/CalypsoAI')
from preprocess import * 

# Load and select learning model hyperparameters

In [4]:
parameters = json.loads(open('/content/gdrive/My Drive/CalypsoAI/config.json').read())

embedding_size = parameters['embedding_dim']
n_classes = parameters['n_classes']
learning_rate = parameters['learning_rate']
n_layers = parameters['n_layers']
lstm_size = parameters['lstm_size']
hidden_unit = parameters['hidden_unit']
epochs = parameters['num_epochs']
batch_size= parameters['batch_size']
dropout = parameters['dropout_keep_prob']
DELTA = parameters['DELTA']

print("LOADED HYPERPARAMETERS")

LOADED HYPERPARAMETERS


# Load and preprocess data

In [5]:
path = '/content/gdrive/My Drive/CalypsoAI/Data/Real or satirical headlines.csv'

# Tokenize headlines and map to one_hot labels
x_, y_, vocabulary,vocabulary_inv,dataframe ,labels, one_hot = load_data(path)
print("===> Loaded and preprocessed Data")
print(one_hot)   
# split data
x, x_test, y, y_test = train_test_split(x_, y_, test_size=0.15, random_state=42)
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.15)

logging.info('x_train: {}, x_dev: {}, x_test: {}'.format(len(x_train), len(x_dev), len(x_test)))
logging.info('y_train: {}, y_dev: {}, y_test: {}'.format(len(y_train), len(y_dev), len(y_test)))

print("LOADED AND PREPROCESSED DATASET")

CRITICAL:root:The maximum length is 63
INFO:root:x_train: 17231, x_dev: 3041, x_test: 3578
INFO:root:y_train: 17231, y_dev: 3041, y_test: 3578


===> Loaded and preprocessed Data
{'genuine': array([1, 0]), 'satire': array([0, 1])}
LOADED AND PREPROCESSED DATASET


# Set sequence length, vocabulary_size and save model path

In [6]:
sequence_length = x_train.shape[1]
vocabulary_size = len(vocabulary)
MODEL_PATH = '/content/gdrive/My Drive/CalypsoAI/model/'

# Define Learning Model (LSTM)

In [7]:
# input, output and dropout placeholders
with tf.name_scope('placeholders'):
  input_x = tf.placeholder(tf.int32, [None, sequence_length], name='input_x')
  input_y = tf.placeholder(tf.float32, [None, n_classes], name='input_y')
  drops = tf.placeholder(tf.float32, name='dropout')
  print("CREATED PLACEHOLDERS")

def get_a_cell(lstm_size, keep_prob):
  lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
  drop_out = tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=drops)
  return drop_out


# Setup the embedding matrix for tensorflow
with tf.variable_scope("embedding"):
    embedding_tf = tf.get_variable("embedding", [vocabulary_size, embedding_size],trainable=True)
    embeddings = tf.nn.embedding_lookup(embedding_tf, input_x)
    print("CREATED EMBEDDINGS")

with tf.name_scope('Lstm'):
  cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell(hidden_unit,drops) for _ in range(n_layers)])
  lstm_outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=embeddings, dtype=tf.float32)
  last_output = state[-1].h
  print("CREATED LSTM")

# preditctions
with tf.name_scope('Fully_connected'):
  W = tf.Variable(tf.truncated_normal([hidden_unit, n_classes], stddev=0.1))
  b = tf.Variable(tf.constant(0.1, shape=[n_classes]))
  output = tf.nn.xw_plus_b(last_output, W, b)
  predictions = tf.argmax(input=output, axis=1, name='predictions')
  print("FULLY CONNECTED")


# Cross-entropy loss and optimizer initialization
with tf.name_scope('Loss'):
  loss = tf.reduce_mean(input_tensor=tf.nn.sigmoid_cross_entropy_with_logits(logits=output, labels=input_y))
  global_step = tf.Variable(0, name="global_step", trainable=False)
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)
  probs = tf.nn.softmax(output)

# Accuracy metrics
with tf.name_scope('Accuracy'):
  correct_pred = tf.equal(tf.argmax(input=input_y, axis=1),predictions)
  accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_pred, tf.float32), name='accuracy')
  tf.summary.scalar('accuracy', accuracy)

with tf.name_scope('correct_predictions'):
  correct_predictions = tf.equal(tf.argmax(input=input_y, axis=1),predictions)
  num_correct = tf.reduce_sum(input_tensor=tf.cast(correct_predictions, 'float'), name='correct_predictions')
  
merged = tf.summary.merge_all()

CREATED PLACEHOLDERS
CREATED EMBEDDINGS
CREATED LSTM
FULLY CONNECTED


# Training process

In [ ]:
# Batch generators
def batch_generator(X, y, batch_size):
    """batch generator"""
    size = X.shape[0]
    X_copy = X.copy()
    y_copy = y.copy()
    indices = np.arange(size)
    np.random.shuffle(indices)
    X_copy = X_copy[indices]
    y_copy = y_copy[indices]
    i = 0
    while True:
        if i + batch_size <= size:
            yield X_copy[i:i + batch_size], y_copy[i:i + batch_size]
            i += batch_size
        else:
            i = 0
            indices = np.arange(size)
            np.random.shuffle(indices)
            X_copy = X_copy[indices]
            y_copy = y_copy[indices]
            continue
            
train_batch_generator = batch_generator(x_train, y_train, batch_size)
test_batch_generator = batch_generator(x_dev, y_dev, batch_size)
predict_generator = batch_generator(x_test, y_test, batch_size)

saver = tf.train.Saver()

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    sess.run(tf.global_variables_initializer())

    print("Start learning...")
    for epoch in range(epochs):
        loss_train = 0
        loss_val = 0
        loss_test = 0
        accuracy_train = 0
        accuracy_val = 0
        accuracy_test = 0
        train_loss_l = []
        val_loss_l = []

        print("epoch: {}\t".format(epoch), end="")

        # Training
        num_batches = x_train.shape[0] // batch_size
        for b in tqdm(range(num_batches)):
            x_batch, y_batch = next(train_batch_generator)
            loss_tr, acc, _, summary = sess.run([loss, accuracy, optimizer, merged],
                                                feed_dict={input_x: x_batch,
                                                           input_y: y_batch,
                                                           drops: dropout})
            train_loss_l.append(loss_tr)
            accuracy_train += acc
            loss_train = loss_tr * DELTA + loss_train * (1 - DELTA)
        accuracy_train /= num_batches
            
        # Validation
        num_batches = x_dev.shape[0] // batch_size
        for b in tqdm(range(num_batches)):
            x_batch, y_batch = next(test_batch_generator)
            val_loss, val_acc, summary = sess.run([loss, accuracy, merged],
                                                     feed_dict={input_x: x_batch,
                                                                input_y: y_batch,
                                                                drops: dropout})
            val_loss_l.append(val_loss)
            accuracy_val += val_acc
            loss_val += val_loss
        accuracy_val /= num_batches
        loss_val /= num_batches

        print("loss: {:.3f}, val_loss: {:.3f}, acc: {:.3f}, val_acc: {:.3f}".format(
            loss_train, loss_val, accuracy_train, accuracy_val))

    # predict x_test
    num_batches = x_test.shape[0] // batch_size
    predict_correct = 0
    for batch in tqdm(range(num_batches)):
        x_batch, yx_batch = next(predict_generator)
        y_true = np.argmax(yx_batch,1)
        loss_pred, acc_pred, n_correct, y_pred = sess.run([loss,accuracy,num_correct,predictions], feed_dict={
            input_x: x_batch,
            input_y: y_batch,
            drops : dropout })
    
    saver.save(sess, MODEL_PATH)
    print("FINISHED TRAINING")     

Start learning...
epoch: 0	

100%|██████████| 23/23 [00:01<00:00, 19.84it/s]


loss: 0.670, val_loss: 0.659, acc: 0.624, val_acc: 0.630
epoch: 1	

100%|██████████| 23/23 [00:00<00:00, 34.44it/s]


loss: 0.487, val_loss: 0.534, acc: 0.713, val_acc: 0.683
epoch: 2	

...

100%|██████████| 23/23 [00:00<00:00, 33.87it/s]


loss: 0.364, val_loss: 0.537, acc: 0.870, val_acc: 0.765
epoch: 55	

100%|██████████| 23/23 [00:00<00:00, 32.88it/s]


loss: 0.381, val_loss: 0.535, acc: 0.871, val_acc: 0.766
epoch: 56	

 70%|██████▉   | 16/23 [00:00<00:00, 34.56it/s]

# Plot training and validation loss

In [ ]:
plt.plot(train_loss_l)
plt.plot(val_loss_l)
plt.show()

# Calculate Precision, Recall and F-1 score to evaluate learning model 

In [ ]:
print("Precision", sk.metrics.precision_score(y_true, y_pred,average='weighted'))
print("Recall", sk.metrics.recall_score(y_true, y_pred,average='weighted'))
print("f1_score", sk.metrics.f1_score(y_true, y_pred,average='weighted'))

# Visualise classifcation results in confusion matrix 

In [ ]:
ax = plt.subplot()
conf_mat = sk.metrics.confusion_matrix(y_true, y_pred)
sns.heatmap(conf_mat, annot=True)
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.xaxis.set_ticklabels(['genuine','satire']); ax.yaxis.set_ticklabels(['genuine','satire']);

# **Additional questions**

### **How might you productize such a model to help in the battle against “fake news”?**

Content verification is already included in a variety of social media platforms, where post descriptions or blog headlines are verified for their truthfulness (e.g.: covid misinformation check). Similarly, such a feature could be used to verify products used in advertisements by influencers. An example of this would be, a product being advertised that is very similar to a well-known brand and indirectly profiting off that reputation.

### **Given your knowledge of the dataset and the model you have created, how would you go about generating satirical headlines to fool your model?**

Unfortunately, there is no one obvious and well-working method of augmenting input data to fool an NLP model like there is in computer vision.  Making NLP , including large language models  (e.g.: BERT or GPT-2) safe from adversarial attacks remains an open problem in NLP and more research is needed. However, there are a few methods that can be used to ‘fool’ a learning model that predicts labels based on textual input sequences. There are two main approaches that have been used in recent research to generate adversarial examples for NLP tasks. The first example of this would be to use visual similarity of a sentence, where the only a few characters in the sentence might be changed or including typos that could have been made by a human writing the headline. One disadvantage of this is that one can easily incorporate automatic spelling checks based on rules or other existing approaches. Another example would be to make the new input semantically as similar as possible to the original input headline (e.g.: paraphrasing the headline so that the meaning remains the same and makes grammatical sense). 